In [122]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import os
import shutil
import json
import geopandas as gpd
import math
from shapely.geometry import Polygon
from PIL import Image

# Comparing rooftop identification

In [51]:
# Load all Google Results into a giant dataframe
basePathGoogle = "/home/jaumeasensio/Documents/Projectes/BEEGroup/solar_potencial_estimation_v3/GoogleAPItests/Results/"

parcelList = []
contructionList = []
planeIDList = []
centroidLatList = []
centroidLonList = []
azimuthList = []
tiltList =  []
areaList = []

for parcel in os.listdir(basePathGoogle):
    for construction in os.listdir(basePathGoogle + parcel):
        responseFile = basePathGoogle + parcel + "/" + construction + "/BuildingInsights.json"
        with open(responseFile, 'r') as file:
            file_content = file.read().replace("'", '"')
        solar_info = json.loads(file_content)

        for idx in range(len(solar_info['solarPotential']['roofSegmentStats'])):
            planeIDList.append(idx)
            centroidLatList.append(solar_info['solarPotential']['roofSegmentStats'][idx]['center']['latitude'])
            centroidLonList.append(solar_info['solarPotential']['roofSegmentStats'][idx]['center']['longitude'])
            azimuthList.append(solar_info['solarPotential']['roofSegmentStats'][idx]['azimuthDegrees'])
            tiltList.append(solar_info['solarPotential']['roofSegmentStats'][idx]['pitchDegrees'])
            areaList.append(solar_info['solarPotential']['roofSegmentStats'][idx]["stats"]['groundAreaMeters2'])

            parcelList.append(parcel)
            contructionList.append(construction)

identificationDF = pd.DataFrame({
    "parcel":parcelList,
    "construction":contructionList,
    "planeID": planeIDList,
    "centroidLatitude": centroidLatList,
    "centroidLongitude": centroidLonList,
    "azimuth": azimuthList,
    "tilt": tiltList,
    "area": areaList
})

duplicates = identificationDF.loc[identificationDF.drop(columns=["construction"]).duplicated(keep='first')]

non_duplicates = identificationDF.loc[~identificationDF.drop(columns=["construction"]).duplicated(keep='first')].reset_index(drop=True).drop(columns=["construction"])
non_duplicates.to_csv("/home/jaumeasensio/Documents/Projectes/BEEGroup/solar_potencial_estimation_v3/Results/Test_70_el Besòs i el Maresme/GoogleComparison/GoogleRooftops.csv", index=False)
non_duplicates

,parcel,planeID,centroidLatitude,centroidLongitude,azimuth,tilt,area
0,4151302DF3845A,0,41.412849,2.211680,225.818400,14.359937,347.30
1,4151302DF3845A,1,41.412901,2.211816,225.723300,14.474403,261.02
2,4151302DF3845A,2,41.412724,2.211336,225.899060,17.038765,154.16
3,4151302DF3845A,3,41.412787,2.211418,225.885180,17.036642,152.83
4,4151302DF3845A,4,41.412849,2.211503,225.751280,16.937689,150.06
...,...,...,...,...,...,...,...
482,4649601DF3844H,94,41.411393,2.216577,245.355870,3.861747,4.89
483,4649601DF3844H,95,41.411218,2.216454,24.604246,4.108605,4.67
484,4649601DF3844H,96,41.411232,2.216420,73.216900,6.636325,4.60
485,4649601DF3844H,97,41.411307,2.216335,46.416584,6.272761,4.58


In [54]:
# Load all my results into a giant dataframe
basePathParcels = "/home/jaumeasensio/Documents/Projectes/BEEGroup/solar_potencial_estimation_v3/Results/Test_70_el Besòs i el Maresme/Parcels/"

parcelList = []
contructionList = []
planeIDList = []
centroidLatList = []
centroidLonList = []
azimuthList = []
tiltList =  []
areaList = []
silhouetteList = []

for parcel in os.listdir(basePathParcels):
    for construction in [x for x in os.listdir(basePathParcels + parcel) if os.path.isdir(basePathParcels + parcel + "/" + x)]:
        planeFile = basePathParcels + parcel + "/" + construction + "/Plane Identification/" + construction + ".gpkg"
        planesGDF = gpd.read_file(planeFile)
        planesGDF = planesGDF.to_crs("EPSG:4326")

        for idx, row in planesGDF.iterrows():
            try:
                centroidLatList.append(row["geometry"].centroid.y)
                centroidLonList.append(row["geometry"].centroid.x)

                parcelList.append(parcel)
                contructionList.append(construction)

                planeIDList.append(row["cluster"])
                
               
                azimuthList.append(row["azimuth"])
                tiltList.append(row["tilt"])
                areaList.append(row["geometry"].area)
                silhouetteList.append(row["silhouette"])
            except:
                print(parcel, construction, idx)

identificationDF = pd.DataFrame({
    "parcel":parcelList,
    "construction":contructionList,
    "planeID": planeIDList,
    "centroidLatitude": centroidLatList,
    "centroidLongitude": centroidLonList,
    "azimuth": azimuthList,
    "tilt": tiltList,
    "area": areaList,
    "silhouette": silhouetteList
})

identificationDF.to_csv("/home/jaumeasensio/Documents/Projectes/BEEGroup/solar_potencial_estimation_v3/Results/Test_70_el Besòs i el Maresme/GoogleComparison/MyRooftops.csv", index=False)
identificationDF
identificationDF

4054901DF3845C 415 1
4054901DF3845C 487 0
4649601DF3844H 546 0


,parcel,construction,planeID,centroidLatitude,centroidLongitude,azimuth,tilt,area,silhouette
0,4151302DF3845A,139,3,41.412830,2.211587,339.609312,0.086,3.129879e-08,0.632105
1,4151302DF3845A,139,1,41.412839,2.211495,143.199484,0.258,1.173136e-07,0.550025
2,4151302DF3845A,139,6,41.412844,2.211548,136.576583,0.192,3.066900e-09,0.784021
3,4151302DF3845A,139,4,41.412836,2.211607,140.146805,0.437,2.110201e-08,0.575725
4,4151302DF3845A,139,2,41.412832,2.211536,317.159530,0.125,7.266388e-08,0.577156
...,...,...,...,...,...,...,...,...,...
176,4649601DF3844H,546,2,41.411159,2.216164,123.383108,0.019,3.748355e-09,0.916273
177,4649601DF3844H,575,2,41.411037,2.215984,48.007441,28.888,9.422149e-10,0.757401
178,4649601DF3844H,575,1,41.411041,2.216004,107.292048,14.147,7.771340e-10,0.675573
179,4649601DF3844H,576,2,41.410986,2.215925,238.617485,12.425,1.767936e-10,0.825521


# Getting Google Panels

In [116]:
def create_output_folder(directory, deleteFolder = False):
    if not(os.path.isdir(directory)):
        os.makedirs(directory)
    else:
        if(deleteFolder):
            shutil.rmtree(directory)
            os.makedirs(directory)

In [106]:
def json2gdf(solar_info):
    solarDF = pd.json_normalize(solar_info['solarPotential']['solarPanels'])

    solarDF["azimuth"] = solarDF["segmentIndex"].apply(
        lambda idx: solar_info['solarPotential']['roofSegmentStats'][idx]["azimuthDegrees"]
    )

    solarDF["tilt"] = solarDF["segmentIndex"].apply(
        lambda idx: solar_info['solarPotential']['roofSegmentStats'][idx]["pitchDegrees"]
    )

    solarDF["height"] = np.where(solarDF["orientation"] == "LANDSCAPE", solar_info["solarPotential"]["panelWidthMeters"], solar_info["solarPotential"]["panelHeightMeters"])
    solarDF["width"] = np.where(solarDF["orientation"] == "LANDSCAPE", solar_info["solarPotential"]["panelHeightMeters"], solar_info["solarPotential"]["panelWidthMeters"])
    solarDF["height"] = solarDF["height"]*np.cos(solarDF["tilt"]*math.pi/180)

    EARTH_RADIUS = 6378137.0

    # Function to calculate corners
    def calculate_corners(row):
        lat = row["center.latitude"]
        lon = row["center.longitude"]
        azimuth = np.radians(row["azimuth"])
        height = row["height"]
        width = row["width"]

        # Calculate dx, dy for height (aligned with azimuth)
        dy_height = (height / 2) * np.cos(azimuth)
        dx_height = (height / 2) * np.sin(azimuth)

        # Calculate dx, dy for width (perpendicular to azimuth)
        dy_width = (width / 2) * np.cos(azimuth + np.pi / 2)
        dx_width = (width / 2) * np.sin(azimuth + np.pi / 2)

        # Convert meters to degrees
        delta_lat_height = dy_height / EARTH_RADIUS * (180 / np.pi)
        delta_lon_height = dx_height / (EARTH_RADIUS * np.cos(np.radians(lat))) * (180 / np.pi)

        delta_lat_width = dy_width / EARTH_RADIUS * (180 / np.pi)
        delta_lon_width = dx_width / (EARTH_RADIUS * np.cos(np.radians(lat))) * (180 / np.pi)

        # Calculate corners
        corners = {
            "sw": (lat - delta_lat_height - delta_lat_width, lon - delta_lon_height - delta_lon_width),
            "se": (lat - delta_lat_height + delta_lat_width, lon - delta_lon_height + delta_lon_width),
            "nw": (lat + delta_lat_height - delta_lat_width, lon + delta_lon_height - delta_lon_width),
            "ne": (lat + delta_lat_height + delta_lat_width, lon + delta_lon_height + delta_lon_width)
        }
        return corners

    # Apply the function to each row
    solarDF["corners"] = solarDF.apply(calculate_corners, axis=1)
    solarDF["latitudes"] = solarDF["corners"].apply(lambda corners: [corners["sw"][0], corners["se"][0], corners["ne"][0], corners["nw"][0], corners["sw"][0]])
    solarDF["longitudes"] = solarDF["corners"].apply(lambda corners: [corners["sw"][1], corners["se"][1], corners["ne"][1], corners["nw"][1], corners["sw"][1]])
    solarDF["geometry"] = solarDF.apply(lambda row: Polygon(zip(row['longitudes'], row['latitudes'])), axis=1)

    gdf = gpd.GeoDataFrame(solarDF, geometry='geometry', crs="EPSG:4326")
    gdf = gdf[["center.latitude", "center.longitude", "yearlyEnergyDcKwh", "geometry"]]
    return gdf

In [120]:
# Load all Google Results into a giant dataframe
basePathGoogle = "/home/jaumeasensio/Documents/Projectes/BEEGroup/solar_potencial_estimation_v3/GoogleAPItests/Results/"
basePathParcels = "/home/jaumeasensio/Documents/Projectes/BEEGroup/solar_potencial_estimation_v3/Results/Test_70_el Besòs i el Maresme/Parcels/"
basePathResults = "/home/jaumeasensio/Documents/Projectes/BEEGroup/solar_potencial_estimation_v3/Results/Test_70_el Besòs i el Maresme/GoogleComparison/"

parcelList = []
contructionList = []
planeIDList = []
centroidLatList = []
centroidLonList = []
azimuthList = []
tiltList =  []
areaList = []

for parcel in os.listdir(basePathGoogle):
    for construction in os.listdir(basePathGoogle + parcel):
        create_output_folder(basePathResults + parcel + "/" + construction + "/Google Panel Generation/")

        cadasterFile = basePathParcels+ parcel + "/" + construction + "/Map files/" + construction + ".gpkg"
        cadasterGDF = gpd.read_file(cadasterFile)

        responseFile = basePathGoogle + parcel + "/" + construction + "/BuildingInsights.json"
        with open(responseFile, 'r') as file:
            file_content = file.read().replace("'", '"')
        solar_info = json.loads(file_content)

        gdf = json2gdf(solar_info)
        gdf.to_crs(cadasterGDF.crs, inplace=True)
        gdf = gdf.clip(mask=cadasterGDF)
        gdf.to_file(basePathResults + parcel + "/" + construction + "/Google Panel Generation/" + construction + ".gpkg")


# fig, ax = plt.subplots()
# gdf.plot(ax=ax, column="yearlyEnergyDcKwh")
# cadasterGDF.plot(ax=ax, edgecolor='black', facecolor='none')
# plt.show()

In [68]:
gdf[["center.latitude", "center.longitude", "yearlyEnergyDcKwh", "geometry"]]

,center.latitude,center.longitude,yearlyEnergyDcKwh,geometry
0,41.412559,2.211390,657.33250,"POLYGON ((2.21139 41.41257, 2.21138 41.41256, ..."
1,41.412256,2.211610,657.06290,"POLYGON ((2.21161 41.41226, 2.2116 41.41225, 2..."
2,41.412485,2.211493,656.51020,"POLYGON ((2.2115 41.41249, 2.21148 41.41248, 2..."
3,41.412423,2.211606,656.35815,"POLYGON ((2.21161 41.41243, 2.21159 41.41242, ..."
4,41.412430,2.211598,656.30910,"POLYGON ((2.2116 41.41244, 2.21159 41.41243, 2..."
...,...,...,...,...
830,41.412571,2.211743,369.75082,"POLYGON ((2.21175 41.41256, 2.21175 41.41257, ..."
831,41.412694,2.211530,368.29678,"POLYGON ((2.21153 41.41269, 2.21154 41.4127, 2..."
832,41.412623,2.211434,367.99374,"POLYGON ((2.21143 41.41261, 2.21145 41.41263, ..."
833,41.412706,2.211546,365.66570,"POLYGON ((2.21154 41.4127, 2.21156 41.41271, 2..."


# Obtaining the flux of the sample points

In [179]:
# Load all Google Results into a giant dataframe
basePathGoogle = "/home/jaumeasensio/Documents/Projectes/BEEGroup/solar_potencial_estimation_v3/GoogleAPItests/Results/"
basePathParcels = "/home/jaumeasensio/Documents/Projectes/BEEGroup/solar_potencial_estimation_v3/Results/Test_70_el Besòs i el Maresme/Parcels/"
basePathResults = "/home/jaumeasensio/Documents/Projectes/BEEGroup/solar_potencial_estimation_v3/Results/Test_70_el Besòs i el Maresme/GoogleComparison/"

parcelList = []
contructionList = []
planeIDList = []
centroidLatList = []
centroidLonList = []
azimuthList = []
tiltList =  []
areaList = []

import tifffile as tiff

for parcel in os.listdir(basePathGoogle):
    for construction in os.listdir(basePathGoogle + parcel):
        create_output_folder(basePathResults + parcel + "/" + construction + "/Google Annual Flux/")
        responseFile = basePathGoogle + parcel + "/" + construction + "/annualFlux.tiff"

        with tiff.TiffFile(responseFile) as tif:
            for i, page in enumerate(tif.pages):
                image = page.asarray()
                
        topY = tif.geotiff_metadata["ModelTransformation"][1][3]
        leftX = tif.geotiff_metadata["ModelTransformation"][0][3]

        PoAfolder = basePathParcels + parcel + "/" + construction + "/Solar Estimation PySAM_POA_Yearly/"
        for file in os.listdir(PoAfolder):
            points = pd.read_csv(PoAfolder + file)
            googleFluxList = []
            for idx, point in points.iterrows():
                pixelX = int((point["x"]-leftX)/0.1)
                pixelY = int((topY-point["y"])/0.1)
                googleFluxList.append(image[pixelX][pixelY])
            points.rename(columns={"annual":"simulatedFlux"})
            points["googleFlux"] = googleFluxList
            points.to_csv(basePathResults + parcel + "/" + construction + "/Google Annual Flux/" + file, index=None)

Page 1 - Shape: (1999, 1996), Data Type: float32


/home/jaumeasensio/anaconda3/envs/qgis_stable/lib/python3.12/site-packages/tifffile/tifffile.py:11426: UserWarning: <tifffile.TiffTag 34735 GeoKeyDirectoryTag @190> reading value from closed file
  warnings.warn(
/home/jaumeasensio/anaconda3/envs/qgis_stable/lib/python3.12/site-packages/tifffile/tifffile.py:11426: UserWarning: <tifffile.TiffTag 34264 ModelTransformationTag @178> reading value from closed file
  warnings.warn(


In [177]:
parcel

'4151302DF3845A'

In [178]:
construction

'139'

In [160]:


int(x-leftX/0.1)
int(topY-y/0.1)

array([[1143.7024 , 1161.9521 , 1181.9557 , ..., 1491.6134 , 1482.6539 ,
        1478.9286 ],
       [1112.3011 , 1145.8661 , 1180.9292 , ..., 1485.7788 , 1480.419  ,
        1479.7666 ],
       [1089.7223 , 1140.4525 , 1184.4585 , ..., 1480.6575 , 1478.7749 ,
        1478.5648 ],
       ...,
       [1415.8367 , 1389.1027 , 1367.4233 , ...,  915.0006 ,  919.71747,
         920.51654],
       [1377.2389 , 1390.4355 , 1389.5833 , ...,  909.7883 ,  911.16626,
         915.35144],
       [1439.022  , 1458.5355 , 1474.4227 , ...,  904.3948 ,  906.0221 ,
         910.9553 ]], dtype=float32)